## Importing the libraries

In [1]:
!pip install torch

  Using cached torch-2.6.0-cp312-cp312-win_amd64.whl.metadata (28 kB)
  Using cached sympy-1.13.1-py3-none-any.whl.metadata (12 kB)
Using cached torch-2.6.0-cp312-cp312-win_amd64.whl (204.1 MB)
   ---------------------------------------- 0.0/6.2 MB ? eta -:--:--
   ---------------------------------------- 0.0/6.2 MB ? eta -:--:--
   - -------------------------------------- 0.3/6.2 MB ? eta -:--:--
   - -------------------------------------- 0.3/6.2 MB ? eta -:--:--
   --- ------------------------------------ 0.5/6.2 MB 1.0 MB/s eta 0:00:06
   ----- ---------------------------------- 0.8/6.2 MB 1.2 MB/s eta 0:00:05
   ------ --------------------------------- 1.0/6.2 MB 1.2 MB/s eta 0:00:05
   ---------- ----------------------------- 1.6/6.2 MB 1.3 MB/s eta 0:00:04
   ----------- ---------------------------- 1.8/6.2 MB 1.4 MB/s eta 0:00:04
   --------------- ------------------------ 2.4/6.2 MB 1.5 MB/s eta 0:00:03
   ---------------- ----------------------- 2.6/6.2 MB 1.5 MB/s eta 0:00:0

In [2]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.optim as optim
import torch.utils.data
from torch.autograd import Variable

D:\anaconda3\Lib\site-packages\torch\utils\_pytree.py:185: FutureWarning: optree is installed but the version is too old to support PyTorch Dynamo in C++ pytree. C++ pytree support is disabled. Please consider upgrading optree using `python3 -m pip install --upgrade 'optree>=0.13.0'`.
  warnings.warn(


## Importing the dataset


enconding - used to import movie names as they contain special characters
header - None as they do not contain header columns

In [3]:
movies=pd.read_csv("ml-1m/movies.dat",sep="::",header=None,engine="python",encoding="latin-1")

In [4]:
movies.head()

,0,1,2
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [5]:
user = pd.read_csv("ml-1m/users.dat",sep="::",header=None,engine="python",encoding="latin-1")

In [6]:
user.head()

,0,1,2,3,4
0,1,F,1,10,48067
1,2,M,56,16,70072
2,3,M,25,15,55117
3,4,M,45,7,02460
4,5,M,25,20,55455


In [7]:
ratings = pd.read_csv('ml-1m/ratings.dat', sep = '::', header = None, engine = 'python', encoding = 'latin-1')

In [8]:
ratings.head()

,0,1,2,3
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


## Preparing the training set and the test set

In [9]:
training_set = pd.read_csv('ml-100k/u1.base', delimiter = '\t')


In [10]:
training_set.head()

,1,1.1,5,874965758
0,1,2,3,876893171
1,1,3,4,878542960
2,1,4,3,876893119
3,1,5,3,889751712
4,1,7,4,875071561


In [11]:
training_set.shape

(79999, 4)

In [12]:
training_set = np.array(training_set, dtype = 'int')
# to work with pytorch tensors we are using array format which is more faster

In [13]:
#same with the test set
test_set = pd.read_csv('ml-100k/u1.test', delimiter = '\t')
test_set = np.array(test_set, dtype = 'int')

In [14]:
test_set.shape

(19999, 4)

## Getting the number of users and movies


In [15]:
nb_users = int(max(max(training_set[:, 0], ), max(test_set[:, 0])))  # for maximum number of users
nb_movies = int(max(max(training_set[:, 1], ), max(test_set[:, 1])))   # For maximum number of movies

In [16]:
nb_movies

1682

In [17]:
nb_users

943

## Converting the data into an array with users in lines and movies in columns


As we are going to work with Torch tensors we are going to create List os Lists not 2d array, as torch expects list of lists

In [18]:
def convert(data):
  new_data = []
  for id_users in range(1, nb_users + 1):
    id_movies = data[:, 1] [data[:, 0] == id_users] # we are accessing the movies_id for each Id_user
    id_ratings = data[:, 2] [data[:, 0] == id_users]   # we are accessing the ratings for movies for each Id_user
    ratings = np.zeros(nb_movies)  # creating a list of zeroes to initialize the ratings to be 0
    ratings[id_movies - 1] = id_ratings  # to assign the ratings from index 0 which is first position
    new_data.append(list(ratings))     # appending the users ratings to whole dataset
  return new_data
training_set = convert(training_set)
test_set = convert(test_set)

## Converting the data into Torch tensors


In [19]:
training_set = torch.FloatTensor(training_set)
test_set = torch.FloatTensor(test_set)

## Converting the ratings into binary ratings 1 (Liked) or 0 (Not Liked)


In [20]:
training_set[training_set == 0] = -1
training_set[training_set == 1] = 0  # or operator doesn't work as usual with torch tensors [training_set == 1 or training_set == 2]
training_set[training_set == 2] = 0
training_set[training_set >= 3] = 1
test_set[test_set == 0] = -1
test_set[test_set == 1] = 0
test_set[test_set == 2] = 0
test_set[test_set >= 3] = 1

## Creating the architecture of the Neural Network


In [21]:
class RBM():
  def __init__(self, nv, nh):  # nv= visible nodes , nh= hidden nodes, _init_ function is the default function
    self.W = torch.randn(nh, nv)  # initializes random value(weight) acc. to normal distribution to tht nodes of matrix(nh,nv) mean =0, variance=1
                                   # as probability of visible nodes acc. to hidden node
    self.a = torch.randn(1, nh)    # adding bias of probability of hidden nodes given that of visible node(a is a parameter of object self)
    self.b = torch.randn(1, nv)    # adding bias of probability of visible nodes given that of hidden node
                                   #it creates a 2D tensor first is for batch size(1,nh),2nd is for bias. Because Pytorch excepts 2D tensor

    

    # for using Gibbs Samplening X is taken for probability for P(h) given P(v)
  def sample_h(self, x):   # This function is for Hidden Nodes
    wx = torch.mm(x, self.W.t())  # product(mm command) of 2 tensors to make vectors(product of weights time neurons plus bias a)
                                       #self.W is used as it is tensors of weight and .t() ,self.W.t() is add to make it transpose to make it correct 
    activation = wx + self.a.expand_as(wx)  # self.a.expand_as(wx), expand is used to add bias are applied to all mimi batches and 
                                              # 1 extra dimension to  make sure it is applied
                                                # a is bias for hidden nodes
    p_h_given_v = torch.sigmoid(activation)  # probability of hidden node getting activated given the value of visible node
    return p_h_given_v, torch.bernoulli(p_h_given_v)  # bernoulli, is used as it is a bernoulli RBM as output is binary 0,1
                                                      # torch.bernoulli is used to activate hidden neurons by bernoulli sampling

    
  def sample_v(self, y):   # This function is for Visible Nodes
    wy = torch.mm(y, self.W)   # here we are not taking Transpose function as it is  already is Probability of visible nodes given Hidden Nodes
    activation = wy + self.b.expand_as(wy)       # taking b as it is bias of visible nodes
    p_v_given_h = torch.sigmoid(activation)            # probability of visibe node(==1) getting activated given the value of hiddden node
    return p_v_given_h, torch.bernoulli(p_v_given_h)     # Number of visible nodes is equal to number opf movies

 #vo= input vector  contains rating, vk= visible nodes after k sampling
 #ph0= probability of hidden node is 1 in 1st itertaion,phk=probalities of hidden node after k sampling given vk


    
  def train(self, v0, vk, ph0, phk): 
    self.W += (torch.mm(v0.t(), ph0) - torch.mm(vk.t(), phk)).t()    # update tensors of weights,mm=product of 2 tensors      
    self.b += torch.sum((v0 - vk), 0)   #we have to keep tensor of 2d
    self.a += torch.sum((ph0 - phk), 0)


#nv= number of visible nodes or number of movies(1 visible node for each movie)
nv = len(training_set[0])
#nh = number of hidden nodes(parameter we choose),like features as actors,directors,gener,producer,action,old,new
nh = 100
# batch size is needed as we update the weight after each observation
batch_size = 32
rbm = RBM(nv, nh)


## Training the RBM

In [22]:
nb_epoch = 10
for epoch in range(1, nb_epoch + 1):  # for any deep learning mmodel we need a loss function
  train_loss = 0  #variable for loss function
  s = 0.  # s is the counter, we will use it to normalize the trainig loss
  for id_user in range(0, nb_users - batch_size, batch_size): # to include users in batch size
    vk = training_set[id_user : id_user + batch_size]  # ratings that alreday existed or given by the user
    v0 = training_set[id_user : id_user + batch_size]  # it is batch of original rating, we will keep this to check with the predictyed value
    ph0,_ = rbm.sample_h(v0) # we have to get our initial probabilities,probabilities that hidden node is 1 given the real ratings,
                             #variable is the visible nodes as we get the probability of hidden node
    for k in range(10):   # it is for k steps for constrative divergence,k steps for round them walk
      _,hk = rbm.sample_h(vk)   # first update the hidden node
      _,vk = rbm.sample_v(hk)    # then update the visible node
      vk[v0<0] = v0[v0<0]        #we dont want to learn(-1)rating as the user have not gviven any ratings,we freeze the ratings of -1,keep -1 as it is
    phk,_ = rbm.sample_h(vk)  # we want to get first element of sample_h
    rbm.train(v0, vk, ph0, phk)
    train_loss += torch.mean(torch.abs(v0[v0 >= 0] - vk[v0 >= 0]))  #we are cal error with simple distance now btw prediction and real rating
    s += 1.                                                          #[v0 >= 0] is given to exclude -1 as it doesnt exist
  print('epoch: '+str(epoch)+' loss: '+str(train_loss/s))


epoch: 1 loss: tensor(0.2958)
epoch: 2 loss: tensor(0.2537)
epoch: 3 loss: tensor(0.2441)
epoch: 4 loss: tensor(0.2507)
epoch: 5 loss: tensor(0.2511)
epoch: 6 loss: tensor(0.2466)
epoch: 7 loss: tensor(0.2490)
epoch: 8 loss: tensor(0.2487)
epoch: 9 loss: tensor(0.2497)
epoch: 10 loss: tensor(0.2497)


In [23]:
test_loss = 0
s = 0.
for id_user in range(nb_users):
    v = training_set[id_user:id_user+1]  # we have to predict for each user,this will be the input,it will activate hidden neurons,its prediction
    vt = test_set[id_user:id_user+1]  # vt is the target or original variable(ratings)
    if len(vt[vt>=0]) > 0:  #we need to make one step,its a blind walk as we predict the next output and filter the non existence
        _,h = rbm.sample_h(v)
        _,v = rbm.sample_v(h)
        test_loss += torch.mean(torch.abs(vt[vt>=0] - v[vt>=0]))
        s += 1.
print('test loss: '+str(test_loss/s))

test loss: tensor(0.2360)
